<a href="https://colab.research.google.com/github/TamannaAhmad/MenstrualCycleDietExercise/blob/main/ProjectCode1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'menstrual-health-and-pcod-risk-detection-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4339065%2F7454556%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240607%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240607T182245Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dae8f5f797b5d7235a4fd080362ad46fd6bb22af39ec1f9774012729850fa8446ee0e066cb43bd841e0c96a99330459cae744386f3a2cb6a71aa38a24b123e39ec9b4216d55bd969563146e61e748ec4a7a23197c06de3b189327297726bf740769bd5212f0b54ea96844cb139c1a5b2f9c4d8c90c13f08f95adc97e7c7beb6097851cb3892a896a81d8835ef9fb35ad76263d49182044a21a32893cc7bc140fb1be8a4b8cda72f64a1e1527ff3e1d9729d12328d148d9ba639b47c6608e541174ce9eeeb36a2695be63af2328ab0edc22f77f59dd641c8278179d364e1a24b3981d42cca48b914bd0135f41f884985fd92106264d2eec292742ac871a5b1b939'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue


[==================================================] 1310 bytes downloaded
Downloaded and uncompressed: menstrual-health-and-pcod-risk-detection-dataset


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df = pd.read_csv("/kaggle/input/menstrual-health-and-pcod-risk-detection-dataset/period - Copy.csv")

In [ ]:
df["Unusual_Bleeding"]=df["Unusual_Bleeding"].map({"NO":0,"No":0,"no":0,"yes":1})
df["Menses_score"]=df["Menses_score"].astype(str).map({"1.0":0, "2.0":1, "3.0":2, "4.0":3, "4.2":4, "4.5":5, "5.0":6})

In [ ]:
df.drop("Income", axis=1, inplace=True)
df.drop("Height", axis=1, inplace=True)

In [ ]:
x=df.drop("Menses_score", axis=1)
y=df["Menses_score"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
dtrain = xgb.DMatrix(x_train,label = y_train)

In [ ]:
dtest = xgb.DMatrix(x_test,label = y_test)

In [ ]:
model = xgb.XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss'
)
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
print(y_pred)
df.columns

[1 0 2 1 1 1 6 0 1 2 1 2 1 2 3 2 6 3 3 2 1 3 3 2 2 3 2 3 2 3 6 2 6]


Index(['number_of_peak', 'Age', 'Length_of_cycle',
       'Estimated_day_of_ovulution', 'Length_of_Leutal_Phase',
       'Length_of_menses', 'Unusual_Bleeding', 'Weight', 'BMI',
       'Mean_of_length_of_cycle', 'Menses_score'],
      dtype='object')

In [ ]:
example_values = [[4,20,26,25,10,6,'no',58,24.2,28]]

In [ ]:
dexample = xgb.DMatrix(example_values)

example_predictions = model.predict(dexample)


In [ ]:
if example_predictions < 3:
  print("Low risk of PCOD")
elif example_predictions == 3:
  print("Moderate risk of PCOD")
elif example_predictions > 3:
  print("High risk of PCOD")

Moderate risk of PCOD
